In [1]:
import sys
sys.path.append('/'.join(sys.path[0].split('/')[:-1]))

import xarray as xr
import numpy as np
import os
from pathlib import Path
import healpy as hp

In [2]:
input_dir = '../data/equiangular/5.625deg/'
output_dir = '../data/healpix/5.625deg/'
nside = 16
n_pixels = 12*(nside**2)

In [9]:
# New HEALPix grid
n_pixels = 12*(nside**2)
hp_lon, hp_lat = hp.pix2ang(nside, np.arange(n_pixels), lonlat=True, nest=True)
lon_idx = xr.DataArray(hp_lon, dims=["lon"])
lat_idx = xr.DataArray(hp_lat, dims=["lat"])

# Interpolate and save all atmospheric fields
all_files = os.listdir(input_dir)
all_files = ['toa_incident_solar_radiation']
for f in all_files:
    # Interpolate
    ds = xr.open_mfdataset(input_dir + f + '/*.nc', combine='by_coords')
    interp_ds = ds.interp(lon=('node', lon_idx), lat=('node', lat_idx)).interpolate_na(dim='node')
    interp_ds = interp_ds.assign_coords(node=np.arange(n_pixels))
    
    # Create out folder
    out_path =  output_dir + f + "/"
    Path(out_path).mkdir(parents=True, exist_ok=True)
    
    # Save
    out_filename = f + '_5.625deg.nc'
    interp_ds.to_netcdf(out_path + out_filename)